# Users and User Roles

::: {.content-hidden when-format="html"}

In [1]:
#| echo: false
#| output: false
import os
os.environ['R_HOME'] = f'C:/Users/{os.environ.get('USERNAME')}/Miniconda3/envs/r_python_jl/Lib/R'

In [2]:
#| echo: false
#| output: false
# enables the %%R magic, not necessary if you've already done this
%load_ext rpy2.ipython
# only have to run once to allow the R magic command

C:\Users\CMD3303\Miniconda3\envs\r_python_jl\Lib\site-packages\rpy2\robjects\packages.py:367: UserWarning: The symbol 'quartz' is not in this R namespace/package.
  warnings.warn(


### Project Setup

::: {.panel-tabset}

#### R

In [3]:
%%capture --no-display 
%%R
library("dplyr")
library("jsonlite")
library("tidyr")
library("REDCapR")
library("knitr")
library("remotes")
library("gt")

In [4]:
%%capture --no-display --no-stdout
%%R

# Detach REDCapR if already loaded, and download the latest version
if (version!='1.1.9005') {
    detach("package:REDCapR", unload=TRUE)
    remotes::install_github("OuhscBbmc/REDCapR")
    library("REDCapR")
    print(packageVersion("REDCapR"))
}

else {
    print("REDCapR package up to date")
}

[1] '1.1.9005'


In [5]:
%%R
# Load API tokens from the json file
token <- jsonlite::fromJSON('./../../json_api_data.json')$dev_token$'309'
url <- "https://dev-redcap.doh.wa.gov/api/"

#### Python

In [6]:
import redcap
import json
import csv
import pandas as pd
import numpy as np
import requests
import tempfile

In [7]:
# Load API tokens from the json file
key = json.load(open('./../../json_api_data.json'))
token = key['dev_token']['309']
url = key['dev_url']

project = redcap.Project(url, token)

:::

:::

REDCap allows strict control of user rights for each project. These rights include a user's ability to edit, export and view data, add or edit reports, import data, create records, etc. Privileges to view and export data are specified for each instrument. 

User roles can be set with predefined privileges and users can then be assigned to these user role groups.

## Users

::: {.panel-tabset}

#### R

Cannot be imported with REDCapR. *See @sec-import_limitations for more information.* 

#### Python

`import_users()`

In this example, we will export the project's users and re-import it so that no changes are made to the project. 

In [8]:
users = project.export_users(format_type='df')
users

,username,email,firstname,lastname,expiration,data_access_group,data_access_group_id,design,alerts,user_rights,...,mobile_app,mobile_app_download_data,record_create,record_rename,record_delete,lock_records_all_forms,lock_records,lock_records_customization,forms,forms_export
0,alexey.gilman@doh.wa.gov,Alexey.Gilman@doh.wa.gov,Alexey,Gilman,NaN,NaN,NaN,1,1,1,...,1,0,1,1,1,0,0,1,"demographics:3,symptoms:1,test_information:1,c...","demographics:1,symptoms:1,test_information:1,c..."
1,caitlin.drover@doh.wa.gov,Caitlin.Drover@doh.wa.gov,Caitlin,Drover,NaN,NaN,NaN,1,1,1,...,1,0,1,1,1,0,0,1,"demographics:3,symptoms:1,test_information:1,c...","demographics:1,symptoms:1,test_information:1,c..."
2,emily.pearman@doh.wa.gov,emily.pearman@doh.wa.gov,Emily,Pearman,NaN,NaN,NaN,1,1,1,...,1,1,1,1,1,0,0,0,"demographics:1,symptoms:1,test_information:1,c...","demographics:1,symptoms:1,test_information:1,c..."


In [9]:
project.import_users(users, import_format='df')

3

**Note:** attemping to import a user already assigned to a user role will result in an error.

:::

## User Roles

::: {.panel-tabset}

#### R

Cannot be imported with REDCapR. *See @sec-import_limitations for more information.* 

#### Python

Roles can be imported using `import_user_roles()` and assigned to a project user using `import_user_role_assignment()`. 

**Importing User Roles**

In [17]:
user_roles = project.export_user_roles(format_type='df')
user_roles

,unique_role_name,role_label,design,alerts,user_rights,data_access_groups,reports,stats_and_charts,manage_survey_participants,calendar,...,mobile_app,mobile_app_download_data,record_create,record_rename,record_delete,lock_records_customization,lock_records,lock_records_all_forms,forms,forms_export
0,U-1564393FT9,Limited Role,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"demographics:0,symptoms:0,test_information:0,c...","demographics:0,symptoms:0,test_information:0,c..."
1,U-5354FA3HYL,Admin,1,1,1,1,1,1,1,1,...,1,0,1,1,1,1,0,0,"demographics:3,symptoms:1,test_information:1,c...","demographics:3,symptoms:1,test_information:1,c..."


**Note:** the `unique_role_name` is automatically generated by REDCap.

In [11]:
project.import_user_roles(user_roles, import_format='df')

3

**Importing User Role Assignments**

In [47]:
user_role_assign = project.export_user_role_assignment(format_type='df')
user_role_assign

,username,unique_role_name,data_access_group
0,alexey.gilman@doh.wa.gov,NaN,NaN
1,caitlin.drover@doh.wa.gov,NaN,NaN
2,emily.pearman@doh.wa.gov,NaN,NaN


In [49]:
user_role_assign['unique_role_name'].astype(str).replace('nan', np.NaN)
user_role_assign.loc[0,'unique_role_name'] = 'U-5354FA3HYL'
user_role_assign

,username,unique_role_name,data_access_group
0,alexey.gilman@doh.wa.gov,U-5354FA3HYL,NaN
1,caitlin.drover@doh.wa.gov,NaN,NaN
2,emily.pearman@doh.wa.gov,NaN,NaN


In [50]:
project.import_user_role_assignment(user_role_assign, import_format='df')

3

::: {.content-hidden when-format="html"}

In [51]:
user_role_assign.loc[0,'unique_role_name'] = np.NaN
user_role_assign

,username,unique_role_name,data_access_group
0,alexey.gilman@doh.wa.gov,NaN,NaN
1,caitlin.drover@doh.wa.gov,NaN,NaN
2,emily.pearman@doh.wa.gov,NaN,NaN


In [52]:
project.import_user_role_assignment(user_role_assign, import_format='df')

3

:::

:::